In [ ]:
import pandas as pd
import sqlite3
import chardet
import time
# coding=utf-8
from konlpy.tag import Hannanum
from konlpy.tag import Twitter
from konlpy.tag import Kkma
hannanum=Hannanum()
twitter=Twitter()
kkma = Kkma()
from apyori import apriori

In [ ]:
con = sqlite3.connect('navernews.db')
df = pd.read_pickle('./loginews.pkl')
df = pd.read_sql("SELECT * FROM logistic", con)

In [ ]:
print(df.info())
df.head()

In [ ]:
df['data'].astype('str')
df['data'] = pd.to_datetime(df['data'], format = "%Y-%m-%d")
df_c =df['content'] 

In [ ]:
for i in range(1,len(df_c)+1):
    a = df_c[i].split()
    df_c[i] = " ".join(a)

In [ ]:
df['content']  = df_c
df[df['content'] =='']
df = df[df['content'] != '']

In [ ]:
df['content'].iloc[0]
print(kkma.pos(df['content'].iloc[0]))

In [ ]:
import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거합니다.
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

In [ ]:
df['ko_text'] = df['content'].apply(lambda x: text_cleaning(x))
df.head()

In [ ]:
df['ko_text'].iloc[0]

In [ ]:
print(kkma.pos(df['ko_text'].iloc[0]))

# EDA

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "./korean_stopwords.txt"
with open(korean_stopwords_path, encoding='utf8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
def get_nouns(x):
    nouns_tagger =Okt()
    nouns = nouns_tagger.nouns(x)
    
    # 한글자 키워드를 제거합니다.
    nouns = [noun for noun in nouns if len(noun) >1]
    
    # 불용어를 제거합니다.
    nouns = [noun for noun in nouns if noun not in stopwords]
    
    
    return nouns

In [ ]:
df['nouns'] = df['ko_text'].apply(lambda x : get_nouns(x))
print(df.shape)
df.head()

In [ ]:
df['nouns'].iloc[0:100]

In [ ]:
df['STD_Y'] =df['data'].dt.strftime('%Y')

In [ ]:
df_2017 = df[df['STD_Y']=='2017']
df_2017 = df_2017[df_2017['class'] != '인사동정']
df_2017

In [ ]:
df_2018 = df[df['STD_Y']=='2018']
df_2018 = df_2018[df_2018['class'] != '인사동정']
df_2018

In [ ]:
df_2019 = df[df['STD_Y']=='2019']
df_2019 = df_2019[df_2019['class'] != '인사동정']
df_2019

In [ ]:
df_2020 = df[df['STD_Y']=='2020']
df_2020 = df_2020[df_2020['class'] != '인사동정']
df_2020

In [ ]:
df_2017['class'].value_counts()

In [ ]:
df_2018['class'].value_counts()

In [ ]:
df_2019['class'].value_counts()

In [ ]:
df_2020['class'].value_counts()

In [ ]:
print(len(df_2017))
print(len(df_2018))
print(len(df_2019))
print(len(df_2020))

In [ ]:
transactions = df_2017['nouns'].tolist()
transactions = [transaction for transaction in transactions  if transaction]# 공백 문자열을 방지합니다.
print(transactions)

In [ ]:
tweet_corpus = "".join(df_2017['ko_text'].tolist())
tweet_corpus

In [ ]:
from konlpy.tag import Okt
from collections import Counter

# 명사 키워드를 추출합니다.
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(tweet_corpus)
count = Counter(nouns)

# 한글자 키워드를 제거합니다.
remove_char_counter = Counter({x : count[x] for x in count if len(x) >1})
print(remove_char_counter)

# TF-IDF

In [ ]:
node_df  =pd.DataFrame(remove_char_counter.items(), columns=['node', 'nodesize'])
node_df  = node_df[node_df['nodesize'] >=50] # 시각화의 편의를 위해 ‘nodesize’ 50 이하는 제거합니다.
node_df.head()

In [ ]:
line_list = df['nouns']

In [ ]:
import gensim
from gensim import corpora
import logging
logging.basicConfig(level=logging.DEBUG)
topic = 5
keyword = 10
texts = []
resultList = []

for line in line_list:
    words = line
    if words != [""]:
        tokens = [word for word in words if (len(word.split("/")[0]) > 1 and word.split("/")[0] not in korean_stopwords_path)]
        texts.append(tokens)
dictionary = corpora.Dictionary(texts)    
corpus = [dictionary.doc2bow(text) for text in texts]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic, id2word = dictionary, passes=10)
for num in range(topic):
    resultList.append(ldamodel.show_topic(num, keyword))

In [ ]:
resultList